In [ ]:
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'

train_data_path = join(data_dir, 'train.csv')
test_data_path = join(data_dir, 'test.csv') 

train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

In [ ]:
#데이터 분리 

# date change
train['date'] = train['date'].apply(lambda i: i[:6]).astype(int)

y = train['price']
del train['price']
del train['id']

# test의 id 컬럼 삭제
test['date'] = test['date'].apply(lambda i: i[:6]).astype(int)

del test['id']

# y log change
y = np.log1p(y)

In [ ]:
#하이퍼파라미터 튜닝

from sklearn.model_selection import RandomizedSearchCV

def my_RandomSearch(model, train, y, param, n_iter= 5, verbose=1, n_jobs=5):
  
    # 1. GridSearchCV 모델로 `model` 초기화
    random_model = RandomizedSearchCV(model, param_distributions=param, \
                        scoring='neg_mean_squared_error', \
                              n_iter =  n_iter, \
                        cv=10, verbose=1, n_jobs=-1)
    
    # 2. 모델을 fitting 
    random_model.fit(train, y)
    
    # 3. params, score에 각 조합에 대한 결과를 저장
    params = random_model.cv_results_['params']
    score = random_model.cv_results_['mean_test_score']
    
    # 4. 데이터 프레임 생성
    results = pd.DataFrame(params)
    results["score"] = score
    
    # 5. RMSLE 값을 추가한 후 점수가 높은 순서로 정렬한 `results`를 반환
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    
    results = results.sort_values("RMSLE")
    return results
#try after getting score which is lower 110000
param = {
    'n_estimators': [int(x) for x in range(370,450,2)],
    'max_depth': [int(x) for x in range(8,10)],}
n_iter = 100

model = LGBMRegressor(random_state=random_state)
my_RandomSearch(model, train, y, param, n_iter, verbose=1) 



In [ ]:
model = LGBMRegressor(max_depth=9, n_estimators=394, random_state=random_state)
model.fit(train, y)

prediction = model.predict(test)
prediction = np.expm1(prediction)
prediction

In [ ]:
array([ 499507.08087058,  499061.13393179, 1374025.54000982, ...,
        471354.80486254,  320740.69734001])

In [ ]:
def save_submission(model, train, y, test, model_name, rmsle=None):
    model.fit(train, y)
    prediction = model.predict(test)
    prediction = np.expm1(prediction)
    data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'
    submission_path = join(data_dir, 'sample_submission.csv')
    submission = pd.read_csv(submission_path)
    submission['price'] = prediction
    submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, model_name, rmsle)
    submission.to_csv(submission_csv_path, index=False)
    print('{} saved!'.format(submission_csv_path))

In [ ]:
save_submission(model, train, y, test, 'lgbm', rmsle = '0.161371')